#Cognitive XAI IMDB Example

Denis Rothman, copyright 2020, MIT License

In [1]:
#@title SHAP Installation
try:
  import shap
except:
  print("Installing shap")
  !pip install shap

Installing shap
     |████████████████████████████████| 276kB 1.4MB/s 
  Created wheel for shap: filename=shap-0.35.0-cp36-cp36m-linux_x86_64.whl size=394118 sha256=e6597d74584bb2b90c4b404473b254616029a27caaa7d0f8747b136a8bc669d1
  Stored in directory: /root/.cache/pip/wheels/e7/f7/0f/b57055080cf8894906b3bd3616d2fc2bfd0b12d5161bcb24ac
Successfully built shap


In [0]:
#@title Import modules
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import numpy as np
import random
import shap

In [0]:
#@title Load IMDB data
corpus,y = shap.datasets.imdb() #importing the data

In [0]:
#@title Split data
sp=0.2 #sample proportion
corpus_train, corpus_test, y_train, y_test = train_test_split(corpus, y, test_size=sp, random_state=7)

In [5]:
#@title Cognitive XAI Policy

pdictionary=["good","excellent","interesting","hilarious","real","great",
             "loved","like","best","cool","adore","impressive","happy",
             "awesome","inspiring","terrific","extraordinary","beautiful",
             "exciting","fascinating","fascinated","pleasure","pleasant",
             "pleasing","pretty","talent","talented","brilliant","genius",
             "bright","creative","fantastic","interesting","recommend",
             "encourage","go","admirable","irrestible","special","unique"]
pl=len(pdictionary)

ndictionary=["bad","worse","horrible","terrible","pathetic","sick","hate",
             "horrific","poor","worst","hated","poorest"]

threshold=len(ndictionary)
y=len(corpus_train)
print("Length",y,"Postive contributions",pl,"Negative contributions",threshold)

tc=0 #true counter
fc=0 #false counter
for i in range(0,y):
  if y_train[i]==True:
    tc+=1
  if y_train[i]==False:
    fc+=1
print("TRAIN: The sample of length",y,"contains", tc,"True values and",fc,"False values")

Length 20000 Postive contributions 40 Negative contributions 12
TRAIN: The sample of length 20000 contains 9926 True values and 10074 False values


In [0]:
#@title Cognitive XAI feature contribution

def cognitive_xai(y,pl,threshold):
  pc=0 #true counter of positive rule
  cc=0   # control counter
  dc=0   # display counter
  show=0 # number of samples to display to control
  for i in range(0,y): 
    fstr=corpus_train[i]
    include=0
    for inc in range(0,pl):
      if fstr.find(pdictionary[inc])>0:
        include=1           
    if pl==0:
      if fstr.find(pdictionary[0])>0:
        include=1           
    exclude=0;
    for inc in range(0,threshold):
      if fstr.find(ndictionary[inc])<0:
        exclude+=1           

    # if-then Rules for true positives
    if include==1:
      if exclude>=threshold:
        cc+=1
        if(y_train[i]==True):
          pc+=1
          if dc< show:dc+=1;print(i,y_train[i],corpus_train[i]);

  print(mcc,"true postives",pc,"scores",round(pc/tc,4),round(pc/cc,4),"TP",tc,"CTP",cc)
  return round(pc/tc,4)

In [7]:
#@title Marginal Cognitive Contribution Metrics
maxpl=pl
for mcc in range(0,pl):
  score=cognitive_xai(y,mcc,threshold)
  if mcc==0:
    print(score,"The MCC is",score,"for",pdictionary[mcc])
    last_score=score
  if mcc>0:
    print(score,"The MCC is",round(score-last_score,4),"for",pdictionary[mcc])
    last_score=score

0 true postives 2484 scores 0.2503 0.667 TP 9926 CTP 3724
0.2503 The MCC is 0.2503 for good
1 true postives 2484 scores 0.2503 0.667 TP 9926 CTP 3724
0.2503 The MCC is 0.0 for excellent
2 true postives 2953 scores 0.2975 0.6938 TP 9926 CTP 4256
0.2975 The MCC is 0.0472 for interesting
3 true postives 3318 scores 0.3343 0.6749 TP 9926 CTP 4916
0.3343 The MCC is 0.0368 for hilarious
4 true postives 3489 scores 0.3515 0.6792 TP 9926 CTP 5137
0.3515 The MCC is 0.0172 for real
5 true postives 4962 scores 0.4999 0.6725 TP 9926 CTP 7378
0.4999 The MCC is 0.1484 for great
6 true postives 5639 scores 0.5681 0.685 TP 9926 CTP 8232
0.5681 The MCC is 0.0682 for loved
7 true postives 5747 scores 0.579 0.6877 TP 9926 CTP 8357
0.579 The MCC is 0.0109 for like
8 true postives 6345 scores 0.6392 0.6737 TP 9926 CTP 9418
0.6392 The MCC is 0.0602 for best
9 true postives 6572 scores 0.6621 0.6778 TP 9926 CTP 9696
0.6621 The MCC is 0.0229 for cool
10 true postives 6579 scores 0.6628 0.6771 TP 9926 CTP 9716

In [8]:
#@title Vectorize datasets
#vectorizing
display=1 #0 no display, 1 display
vectorizer = TfidfVectorizer(min_df=1000,lowercase=False)

X_train = vectorizer.fit_transform(corpus_train)

#visualizing the vectorized features
feature_names=vectorizer.get_feature_names()
lf=(len(feature_names))
print("Number of features",lf)
if display==1:
  for fv in range(0,lf):
    print(feature_names[fv],round(vectorizer.idf_[fv],5))

Number of features 434
10 3.00068
After 3.85602
All 3.5751
American 3.74347
And 2.65868
As 3.1568
At 3.84909
But 2.55358
DVD 3.60509
Even 3.82267
For 3.60374
He 2.96224
Hollywood 3.83963
However 3.66792
If 2.72602
In 2.69097
It 1.78982
John 3.76865
My 3.59969
No 3.87622
Not 3.59634
One 3.50598
She 3.55753
So 3.4019
TV 3.51707
That 3.37521
The 1.34437
There 2.6274
They 3.2692
This 1.83453
To 3.87445
We 3.69494
Well 3.88872
What 3.1813
When 3.4758
You 3.21783
about 1.87628
absolutely 3.98583
acting 2.56643
action 3.41357
actor 3.57839
actors 2.93314
actually 3.03108
after 2.65109
again 3.09318
all 1.69932
almost 3.28873
along 3.83281
also 2.49571
although 3.89503
always 3.2591
am 3.41468
an 1.73694
and 1.03978
another 3.07271
any 2.4917
anyone 3.4628
anything 3.30865
are 1.5919
around 3.12323
as 1.47825
at 1.70527
audience 3.62215
away 3.35393
awful 3.87445
back 2.87671
bad 2.4789
be 1.58014
beautiful 3.66936
because 2.41742
become 3.90777
been 2.32657
before 3.01071
beginning 3.96622
be

In [9]:
#@title Cognitive Min Vectorizing Control
lf=(len(feature_names))
if display==1:
  print("Positive contributions:")
  for fv in range(0,lf):
    for check in range (0,pl):
      if(feature_names[fv]==pdictionary[check]):
        print(feature_names[fv],round(vectorizer.idf_[fv],5),)
  print("\n")
  print("Negative contributions:")
  for fv in range(0,lf):
    for check in range (0,threshold):
      if(feature_names[fv]==ndictionary[check]):
        print(feature_names[fv],round(vectorizer.idf_[fv],5))

Positive contributions:
beautiful 3.66936
best 2.6907
excellent 3.71211
go 2.84016
good 1.98315
great 2.43092
interesting 3.27017
interesting 3.27017
like 1.79004
pretty 3.1853
real 2.94846
recommend 3.75755
special 3.69865


Negative contributions:
bad 2.4789
poor 3.82351
terrible 3.95368
worst 3.46163
